# SENTIMENT ANALYSIS

## Importing Libraries

In [47]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [48]:
REPLACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

## Load Dataset

### Training Set

In [49]:
training_set = []
for line in open('data/movie_data/full_train.txt', 'r', encoding = "utf8"):
	training_set.append(line.strip())
print(training_set[0:10])

['Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!', 'Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from

In [50]:
testing_set = []
for line in open('data/movie_data/full_test.txt', 'r', encoding = "utf8"):
	testing_set.append(line.strip())
print(testing_set[0:10])

["I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.", 'Actor turned director Bill Paxton follows up his promising debut, the Gothic-horror "Frailty", with this family friendly sports drama about the 1913 U.S. Open where a young American caddy rises from h

## Data Cleaning

### Training Set Cleaning 

In [51]:
training_set = [REPLACE.sub("", line.lower()) for line in training_set]
training_set = [REPLACE_SPACE.sub(" ", line) for line in training_set]

### Testing Set Cleaning

In [52]:
testing_set = [REPLACE.sub("", line.lower()) for line in testing_set]
testing_set = [REPLACE_SPACE.sub(" ", line) for line in testing_set]

## Train Model

### Count Vectorizer Model

In [53]:
cv = CountVectorizer(binary=True)
cv.fit(training_set)
X = cv.transform(training_set)
X_test = cv.transform(testing_set)
target = [1 if i < 12500 else 0 for i in range(25000)]

## Splitting Data

In [54]:
x_train, x_test , y_train, y_test = train_test_split(X, target, train_size = 0.75)

### Logistic Regression Model

## Calculating Accuracy_Score

In [55]:
score = 0
for c in [0.01, 0.05, 0.25, 0.50, 0.75, 1.00]:
	lr = LogisticRegression(C=c,max_iter=100000)
	lr.fit(x_train, y_train)
	print ("Accuracy for C=%s: %s" 
	       % (c, accuracy_score(y_test, lr.predict(x_test))))
	if(score < accuracy_score(y_test, lr.predict(x_test))):
	    score = accuracy_score(y_test, lr.predict(x_test))

Accuracy for C=0.01: 0.87408
Accuracy for C=0.05: 0.88144
Accuracy for C=0.25: 0.8752
Accuracy for C=0.5: 0.87216
Accuracy for C=0.75: 0.87056
Accuracy for C=1.0: 0.86928


In [56]:
output_model = LogisticRegression(C=score, max_iter=10000)
output_model.fit(X, target)

LogisticRegression(C=0.88144, max_iter=10000)

## Output Model Accuracy 

In [57]:
accuracy = accuracy_score(target, output_model.predict(X_test))
percentage_accuracy = accuracy*100.00
print("Output Model Accuracy : %0.2f" % percentage_accuracy + "%")

Output Model Accuracy : 87.00%


## Examples

In [58]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), output_model.coef_[0]
    )
}

In [59]:
print("Example of Positive words and it's Weightage")
for positive in sorted(
	feature_to_coef.items(), 
	key=lambda x: x[1], 
	reverse=True)[:5]:
	print (positive)

Example of Positive words and it's Weightage
('excellent', 1.5306374623466603)
('refreshing', 1.4784513515047977)
('perfect', 1.3519875893817983)
('appreciated', 1.3412216973405184)
('superb', 1.3045359057138057)


In [60]:
print("Example of Negative words and it's Weightage")
for negative in sorted(
	feature_to_coef.items(), 
	key=lambda x: x[1])[:5]:
	print (negative)

Example of Negative words and it's Weightage
('worst', -2.2977923978191925)
('waste', -2.136371368258937)
('disappointment', -1.9409072994265748)
('poorly', -1.9171757937288785)
('awful', -1.719876060800618)


## Test Human Generated Preview 

In [ ]:
predictions = output_model.predict(cv.transform([input("Enter Your Own Review :")]))[0]
if(predictions == 0):
	print("Negative Review!(-)")
else:
	print("Positive Review!(+)")